<h1 style='text-align:center'>Homework 6</h1>
<h2 style='text-align:center'>Caleb Eades</h2>
<h2 style='text-align:center'>Due Date: April 2, 2018</h2>

<h3>Notes:</h3>
<p>
<ul>
<li>I use some globals in here to keep the name_map (mapping of category numbers to actual category names) and classifer (dictionary of the StandardScaler and RandomForestClassifier) objects readily available later.</li>
<li>I didn't have time to run the training model using cross-validation so I have an accuracy of 100% on every category I tested which obviously means it doesn't mean much for future predictive power.</li>
<li>Also, a big one, apparently my correlation features have 0 feature importance in the random forest, so either I implemented them wrong (which I don't think I did based on preliminary plots and tests on the images) or they are just extremely useless features (which I also kind of doubt). So there's probably a bug somewhere, but my forest still predicts the training data 100% (again a useless statistic other than to say that it works on the training set and isn't being completely guessing or anything). So given more time, I would probably seek out other features to replace these correlation ones or try to fix whatever weird bug is happening here.</li>
</ul>
</p>

In [1]:
import os
from scipy.misc import imread

def get_tail(string):
    """
    Helper function for the get_subdirs function. It takes a directory
    string and gives the name after the last backslash, thereby giving
    the final subdirectory.
    
    :string: (str) string representing a specific directory
    
    :returns: (str) the name of the final subdirectory
    """
    splits = string.split('/')
    return splits[-1]

def get_subdirs():
    """
    Gets the subdirectories in the 50_categories folder where all the data
    is stored.
    
    :returns: (list of str) the list of all the categories of images
    """
    cwd = os.getcwd()
    category_dir = cwd + '/50_categories'
    # note the [1:] in the next line because for some reason os.walk gives
    # the root directory as the first element, which we don't care about
    subdirs = [get_tail(x[0]) for x in os.walk(category_dir)][1:]
    return subdirs

def get_image_paths(subdir_name):
    """
    Gets all the images from the specified subdirectory of 50_categories.
    
    :subdir_name: (str) the name of the subdirectory (from get_subdirs)
    
    :returns: (list of str) list of all the direct image paths in the subfolder
    """
    cwd = os.getcwd()
    category_dir = cwd + '/50_categories'
    # note the [1:] in the next line because for some reason os.walk gives
    # the root directory as the first element, which we don't care about
    return os.listdir(category_dir + '/' + subdir_name)

def convert_images():
    categories = get_subdirs()
    category_map = {}
    images = {}
    for i,name in enumerate(categories):
        category_map[i] = name
        images[i] = [imread('./50_categories/{}/'.format(name) + image_name) for image_name in get_image_paths(name)]
    return category_map,images

name_map,images = convert_images()

/home/kceades/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [2]:
import numpy as np
from skimage.feature import structure_tensor,CENSURE

def rgb_corr_features(image):
    """
    Takes in an image (as a numpy array) and calculates correlations between
    the rgb data and returns this as a tuple.
    
    :image: (numpy array, NxMx3) the input image
    
    :returns: (list of floats) correlation tuple (rg,rb,gb)
    """
    try:
        r = image[:,:,0]
        r = r.reshape(len(r)*len(r[0]))
        g = image[:,:,1]
        g = g.reshape(len(g)*len(g[0]))
        b = image[:,:,2]
        b = b.reshape(len(b)*len(b[0]))
        adj_r = preprocessing.scale(r)
        adj_g = preprocessing.scale(g)
        adj_b = preprocessing.scale(b)
        rg = np.corrcoef(adj_r,adj_g)[0][1]
        rb = np.corrcoef(adj_r,adj_b)[0][1]
        gb = np.corrcoef(adj_g,adj_b)[0][1]
        return [rg,rb,gb]
    except:
        return [0,0,0] # default to no correlation
    
def aspect_ratio(image):
    """
    Takes in an image (as a numpy array) and calculates the ratio of the
    width to the height.
    
    :image: (numpy array, NxMx3) the input image
    
    :returns: (list of float) the ratio width/height
    """
    try:
        return [len(image[0])/len(image)]
    except:
        return [1] # default to square image

def mean_color_features(image):
    """
    Takes in an image (as a numpy array) and calculates the mean of each
    color in the image and returns this as a tuple.
    
    :image: (numpy array, NxMx3) the input image
    
    :returns: (list of floats) mean of the colors (r,g,b)
    """
    try:
        r_mean = np.mean(image[:,:,0])
        g_mean = np.mean(image[:,:,1])
        b_mean = np.mean(image[:,:,2])
        return [r_mean,g_mean,b_mean]
    except:
        return [122.5,122.5,122.5] # default to mean colors of half 255
    
def structure_tensor_features(image):
    """
    Takes in an image (as a numpy array) and calculates the mean and std of each
    Axx, Axy, and Ayy structure tensors and returns this as a tuple.
    
    :image: (numpy array, NxMx3) the input image
    
    :returns: (list of floats) mean and std of the structure tensors
                with (Axx mean, Axx std, Axy mean, Axy std, Ayy mean, Ayy std)
    """
    try:
        Axx,Axy,Ayy = structure_tensor(np.mean(image,axis=2))
        return [np.mean(Axx),np.std(Axx)\
               ,np.mean(Axy),np.std(Axy)\
               ,np.mean(Ayy),np.std(Ayy)]
    except:
        return [0,0,0,0,0,0] # default to no structure and no variance
    
def censure_features(image):
    """
    Takes in an image (as a numpy array) and calculates the mean and std
    of the scales of the keypoints and returns this as a tuple.
    
    :image: (numpy array, NxMx3) the input image
    
    :returns: (list of floats) mean and std of the CENSURE scales
    """
    try:
        censure = CENSURE()
        censure.detect(np.mean(image,axis=2))
        scales = censure.scales
        return [np.mean(scales),np.std(scales)]
    except:
        return [0,0] # default to no keypoints

In [3]:
feature_map = {'censure':censure_features\
              ,'structure':structure_tensor_features\
              ,'mean_color':mean_color_features\
              ,'aspect':aspect_ratio\
              ,'color_corr':rgb_corr_features}

def get_features(image):
    """
    Gets all the features of the images and returns them in an array
    together using the feature methods written above.
    
    :image: (numpy array, NxMx3) the input image
    
    :returns: (list of floats) the features
    """
    final_features = []
    for feature in feature_map:
        final_features.extend(feature_map[feature](image))
    return final_features

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

def create_classifier():
    """
    Creates the random forest classifier using the features written
    above.
    
    :returns: (tuple) tuple of (StandardScaler,RandomForestClassifier) objects
    """
    X_train = []
    Y_train = []
    for key in name_map:
        for image in images[key]:
            X_train.append(get_features(image))
            Y_train.append(key)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    forest = RandomForestClassifier(n_estimators=100)
    forest.fit(X_train,Y_train)
    return scaler,forest

Note here in the line below that I created the classifier with a block of code such as:

s,f = create_classifier()

and then dumped a dictionary of {'scaler':s,'classifier':f} into a pickled file so I could load it up quickly later rather than regenerating every time I opened up the notebook.

In [5]:
import pickle
class_file = open('classifier.p','wb')
s,f = create_classifier()
pickle.dump({'scaler':s,'classifier':f},class_file)
class_file.close()

In [ ]:
class_file = open('classifier.p','rb')
classifier = pickle.load(class_file)
class_file.close()

Based on the results below, the most important features are (in order):
<ol>
<li>Standard deviation of the Axy structure tensor feature.</li>
<li>Standard deviation of the Axx structure tensor feature.</li>
<li>The mean blue color of the image.</li>
</ol>
Note that I got this because I inputted features into the classifier in the order of:
<ol>
<li>Censure features (2 of them)</li>
<li>Structure tensor features (6 of them)</li>
<li>Mean color features (3 of them)</li>
<li>Aspect feature (1 of them)</li>
<li>Color correlation feature (3 of them)</li>
</ol>
The exact features are detailed above in the input cell that has all the feature calculations.

In [21]:
classifier.keys()
print(classifier['classifier'].feature_importances_)
for feature in feature_map:
    print(feature)

[ 0.07325902  0.07002834  0.07813482  0.07871519  0.06936508  0.08177909
  0.07728593  0.0732853   0.07578348  0.07621652  0.07867366  0.16747358
  0.          0.          0.        ]
censure
structure
mean_color
aspect
color_corr


In [7]:
def test_on_training():
    """
    Tests the classifier on the training sample to see
    how much better it did than random guessing (I realize
    this isn't cross validation but my classifier took
    a long time to build and I didn't have time to wait
    for each run in a full cross validation).
    """
    successes = 0
    failures = 0
    for key in name_map:
        for image in images[key]:
            X_test = get_features(image)
            scaled = classifier['scaler'].transform([X_test])
            category = classifier['classifier'].predict(scaled)
            if category[0]==key:
                successes += 1
            else:
                failures += 1
        break # only testing on the first category
    print('Error rate of {:.2f} percent.'.format(failures*100/(successes+failures)))
    print('Random guessing error rate: 98 percent')

As detailed above in the docstring, the following line doesn't really mean much: it just says that the error rate is 0 (on only the first category) because I'm testing on the training sample that was used to generate the classifier. I didn't have enough time to do full k-fold cross validation based on the amount of time it was taking to get the features and build the classifier for each run.

In [8]:
test_on_training()

Error rate of 0.00 percent.
Random guessing error rate: 98 percent


In [17]:
def run_final_classifier(directory):
    in_dir = os.listdir(directory)
    files = [file for file in in_dir if os.path.isfile(os.path.join(directory,file))]
    images = []
    for file in files:
        images.append((file,imread(os.path.join(directory,file))))
    log_file = open('log_file.txt','w')
    log_file.write('File Name \t\t\t Classification \n')
    for image in images:
        X = get_features(image[1])
        scaled = classifier['scaler'].transform([X])
        category = classifier['classifier'].predict(scaled)[0]
        log_file.write(image[0] + ' \t\t\t {} \n'.format(name_map[category]))
    log_file.close()

The following line tests the log file creation using one of the given categories, which it does work.

In [18]:
run_final_classifier('./50_categories/dog')

/home/kceades/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
